In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from grakel import GraphKernel
from grakel import Graph

import random
from random import shuffle

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

from collections import defaultdict
from itertools import chain, groupby
from tqdm import tqdm
import json
import matplotlib.pylab as plt
import time

plt.rcParams['figure.figsize'] = [15, 15]

In [2]:
def matrix_prod_trace(A,B):
    n = len(A)
    diag_sum = 0
    for i in range(n):
        diag_sum += np.matmul(A[i,:],B[:,i])
    return diag_sum

def hash_p_stable(projection, r):
    bin_counts = {}
    for i in range(1):
        b = np.random.uniform(0,r)
        bin_hash = int(np.floor((projection + b ) / r ))
        bin_counts[bin_hash] = bin_counts.get(bin_hash, 0) + 1

    max_value = max(bin_counts, key=bin_counts.get)
    return max_value

def get_erdos_renyi_graph(n,p):
    done = True
    while done:
        g = nx.erdos_renyi_graph(n,p)
        if g.number_of_edges() > 0:
            return g

# Generate synthetic graphs

In [3]:
num_graph = 1000
n_nodes = 20 # 5, 7, 10, 12, 15, 20
m_ba = 3
dim = 2

target_avg_degree = 2 * m_ba
p_target = target_avg_degree / (n_nodes - 1)  # edge probability approximation
r = np.sqrt(p_target / np.pi)           # radius heuristic in 2D

seed=42

In [4]:
rgg_graphs = []
i = 0
while len(rgg_graphs) < int(num_graph/2):
    #g = nx.erdos_renyi_graph(n=n_nodes, p=p_er, seed=seed)
    g = nx.random_geometric_graph(n=n_nodes, radius=r, dim=dim, seed=seed)
    if g.number_of_edges() > 0:
        rgg_graphs.append(g)
    i += 1

In [5]:
ba_graphs = []
i = 0
while len(ba_graphs) < int(num_graph/2):
    g = nx.barabasi_albert_graph(n=n_nodes, m=m_ba, seed=seed)
    if g.number_of_edges() > 0:
        ba_graphs.append(g)
    i += 1

In [6]:
graphs = rgg_graphs + ba_graphs
y = np.zeros(num_graph)

In [7]:
len(graphs)

1000

In [8]:
y[:int(num_graph/2)] = 1

In [9]:
graphs, y = shuffle(graphs, y)

In [10]:
graphs_train, graphs_test, y_train, y_test = train_test_split(graphs, y, test_size=0.2, random_state=42)

# Generate Buckets

In [11]:
start_time = time.time()

In [12]:
%%time

num_proj = 5
B = 100
num_graph = len(graphs_train)

projections = []
bucket_ids = [''] * num_graph
for k in tqdm(range(num_proj)):
    erg = get_erdos_renyi_graph(n_nodes, 0.8)
    A_er = nx.adjacency_matrix(erg)
    
    for i in range(num_graph):
        A = nx.adjacency_matrix(graphs_train[i])
        proj = matrix_prod_trace(A.toarray(),A_er.toarray())
        projections.append(proj)

    min_proj = min(projections)
    max_proj = max(projections)
    r = (max_proj - min_proj)/B
    for i in range(num_graph):
        hash_bin = hash_p_stable(projections[i], r)
        bucket_ids[i] += str(hash_bin)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.20it/s]

CPU times: user 806 ms, sys: 24.6 ms, total: 831 ms
Wall time: 809 ms


# Choose representative by bucket

In [13]:
clusters = defaultdict(list)
for idx, label in enumerate(bucket_ids):
    clusters[label].append(idx)

idx = [random.choice(indices) for indices in clusters.values()]
len(idx)

31

In [14]:
representatives = [Graph(nx.to_dict_of_lists(graphs_train[i])) for i in idx]

In [15]:
len(representatives)

31

In [16]:
y_train2 = [y_train[i] for i in idx]

In [17]:
from collections import Counter
Counter(y_train2)

Counter({1.0: 16, 0.0: 15})

# Create Kernel Matrix

In [18]:
%%time
kernel = GraphKernel(kernel={"name": "random_walk"}, normalize=False)
#kernel = GraphKernel(kernel={"name": "shortest_path"}, normalize=True)
K_train = kernel.fit_transform(representatives)

CPU times: user 332 ms, sys: 1.3 ms, total: 333 ms
Wall time: 332 ms


# Train 

In [19]:
%%time
clf = SVC(kernel="precomputed")
clf.fit(K_train, y_train2)

CPU times: user 1.72 ms, sys: 0 ns, total: 1.72 ms
Wall time: 1.37 ms


SVC(kernel='precomputed')

In [20]:
print(" Training time --- %s seconds ---" % (time.time() - start_time))

 Training time --- 1.2286555767059326 seconds ---


In [21]:
clf.n_support_

array([1, 1], dtype=int32)

In [22]:
sum(clf.n_support_)

2

# Prediction over test

In [23]:
start_time = time.time()

In [24]:
graphs_test2 = [Graph(nx.to_dict_of_lists(graphs_test[i])) for i in range(len(graphs_test))]
K_test = kernel.transform(graphs_test2)

In [25]:

y_pred = clf.predict(K_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [26]:
print(" Prediction time --- %s seconds ---" % (time.time() - start_time))

 Prediction time --- 3.9825599193573 seconds ---


# Comple Training

In [27]:
start_time = time.time()

In [28]:
graphs_train2 = [Graph(nx.to_dict_of_lists(graphs_train[i])) for i in range(len(graphs_train))]

In [29]:
%%time
kernel = GraphKernel(kernel={"name": "random_walk"}, normalize=False)
#kernel = GraphKernel(kernel={"name": "shortest_path"}, normalize=True)
K_train = kernel.fit_transform(graphs_train2)

CPU times: user 3min 30s, sys: 21.4 ms, total: 3min 30s
Wall time: 3min 30s


In [30]:
%%time
clf = SVC(kernel="precomputed")
clf.fit(K_train, y_train)

CPU times: user 2.53 ms, sys: 1.02 ms, total: 3.55 ms
Wall time: 1.96 ms


SVC(kernel='precomputed')

In [31]:
clf.n_support_

array([1, 1], dtype=int32)

In [32]:
sum(clf.n_support_)

2

In [33]:
print(" Training time --- %s seconds ---" % (time.time() - start_time))

 Training time --- 211.05767726898193 seconds ---


# Prediction over test

In [34]:
start_time = time.time()

In [35]:
graphs_test2 = [Graph(nx.to_dict_of_lists(graphs_test[i])) for i in range(len(graphs_test))]
K_test = kernel.transform(graphs_test2)

In [36]:
y_pred = clf.predict(K_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [37]:
print(" Prediction time --- %s seconds ---" % (time.time() - start_time))

 Prediction time --- 108.0149405002594 seconds ---
